In [116]:
using Distributions, Compat, Plots, Random, CSV, DataFrames, GLM, Printf
plotly()
upscale = 1
default(line = (:line, 0.5, 1, 2), size=(800*upscale,600*upscale))

In [133]:
include("../KalmanFilter.jl")
import .KalmanFilter

### Daten für die folgende Experimente

In [134]:
data_scale = 0.01
x = collect(0:data_scale:10)
y = sin.(x)
n = Normal(0, 0.1)
y_measurement = map(x -> x - rand(n), y)
plot(x, y_measurement, ylim = (-2, 2), label = "Generierte Daten mit Rauschen")
plot!(x, y, label = "Daten ohne Rauschen")
plot!(x, abs.(y - y_measurement), label = "Messfehler", linestyle = :dot, line = :steppre)

---
### Experiment: Filterung reiner Sinus Werte
Wir versuchen, nur das original Signal zu verfolgen, also diese ohne Rauschen.

In [135]:
A = 1 # Zustandsübergangsmatrix
G = 1 # Messabbildung Matrix
Q = .8
R = 0.5

x̂_0 = 0
# x̂_0 = f(1)
Σ_0 = 0.5

model = KalmanFilter.Kalman(A, Q, G, R, x̂_0, Σ_0)

y_filtered = zeros(length(x))
y_predicted = zeros(length(x))
for (index, value) in enumerate(x)
    y_cur = y[index]
    y_filtered[index] = KalmanFilter.fusion(model, y_cur).state
    y_predicted[index] = KalmanFilter.predict(model).state
    #@printf "Step %4d    y_m %.3f    filtered %.3f    predicted %.3f    gain %.3f\n" index y_cur y_filtered[index] y_predicted[index] KalmanFilter.K(model)
end

plot(x, y_predicted, label = "Predicted")
plot!(x, y_filtered, label = "Filtered", linestyle=:dash)
plot!(x, y, label = "Ground Thruth", linestyle=:dot)

---
### Experiment: Filterung sinus mit Rauschen

Wir versuchen das Originale Signal herzustellen und lassen den Kalman Filter über das Datensatz mit rauschen laufen.

In [136]:
A = 1 # Zustandsübergangsmatrix
G = 1 # Messabbildung Matrix
Q = .8
R = 0.5

x̂_0 = 0
# x̂_0 = f(1)
Σ_0 = 0.5

model = KalmanFilter.Kalman(A, Q, G, R, x̂_0, Σ_0)

y_filtered = zeros(length(x))
y_predicted = zeros(length(x))
for (index, value) in enumerate(x)
    y_cur = y_measurement[index]
    y_filtered[index] = KalmanFilter.fusion(model, y_cur).state
    y_predicted[index] = KalmanFilter.predict(model).state
    #@printf "Step %4d    y_m %.3f    filtered %.3f    predicted %.3f    gain %.3f\n" index y_cur y_filtered[index] y_predicted[index] KalmanFilter.K(model)
end

plot(x, y_predicted, label = "Predicted")
plot!(x, y_filtered, label = "Filtered", linestyle=:dash)
plot!(x, y_measurement, label = "Measurement")
plot!(x, y, label = "Ground Thruth", linestyle=:dot)

#### Ergebnis

---
### Experiment: Sinus Werte ohne Rauschen mit Steuerungssignal
Wir lassen den Kalman Filter über reiner Sinus laufen, verwenden aber ein Steuerungssignal.

In [142]:
A = 1 # Zustandsübergangsmatrix
G = 1 # Messabbildung Matrix
Q = .8
R = 0.5
B = 0.01
x̂_0 = 0
# x̂_0 = f(1)
Σ_0 = 0.5

model = KalmanFilter.Kalman(A, Q, G, R, x̂_0, Σ_0, B, 1)

y_filtered = zeros(length(x))
y_predicted = zeros(length(x))
for (index, value) in enumerate(x)
    y_cur = y[index]
    y_filtered[index] = KalmanFilter.fusion(model, y_cur).state
    y_predicted[index] = KalmanFilter.predict(model, sin(value)).state
    #@printf "Step %4d    y_m %.3f    filtered %.3f    predicted %.3f    gain %.3f\n" index y_cur y_filtered[index] y_predicted[index] KalmanFilter.K(model)
end

plot(x, y_predicted, label = "Predicted")
plot!(x, y_filtered, label = "Filtered", linestyle=:dash)
plot!(x, y, label = "Ground Thruth", linestyle=:dot)

#### Ergebnis


---
### Experiment: Sinus Werte mit Rauschen und Steuerungssignal

In [143]:
A = 1 # Zustandsübergangsmatrix
G = 1 # Messabbildung Matrix
Q = .2
R = 1
B = sin
x̂_0 = 0
# x̂_0 = f(1)
Σ_0 = 0.5

model = KalmanFilter.Kalman(A, Q, G, R, x̂_0, Σ_0)

y_filtered = zeros(length(x))
y_predicted = zeros(length(x))
for (index, value) in enumerate(x)
    y_cur = y_measurement[index]
    y_filtered[index] = KalmanFilter.fusion(model, y_cur).state
    y_predicted[index] = KalmanFilter.predict(model, data_scale * sin(value)).state
    #@printf "Step %4d    y_m %.3f    filtered %.3f    predicted %.3f    gain %.3f\n" index y_cur y_filtered[index] y_predicted[index] KalmanFilter.K(model)
end

plot(x, y_predicted, label = "Predicted")
plot!(x, y_filtered, label = "Filtered", linestyle=:dash)
plot!(x, y_measurement, label = "Measurement")
plot!(x, y, label = "Ground Thruth", linestyle=:dot)

In [102]:
describe(x)

Summary Stats:
Mean:           10.000000
Minimum:        0.000000
1st Quartile:   5.000000
Median:         10.000000
3rd Quartile:   15.000000
Maximum:        20.000000
Length:         41
Type:           Float64
